In [1]:
from transformers import *

In [11]:
import tokenizers

In [14]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file='/Users/ankur.kumar/Downloads/vocab.json', 
    merges_file="/Users/ankur.kumar/Downloads/merges.txt", 
    lowercase=True,
    add_prefix_space=True
)

In [3]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading: 100%|██████████| 232k/232k [00:01<00:00, 190kB/s]  


In [15]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }

In [16]:
process_data('Sooo SAD I will miss you here in San Diego!!!','Sooo SAD','negative',tokenizer, 120)

{'ids': [0,
  2430,
  2,
  2,
  98,
  3036,
  5074,
  939,
  40,
  2649,
  47,
  259,
  11,
  15610,
  1597,
  2977,
  16506,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [21]:
tweet = 'Sooo SAD I will miss you here in San Diego!!!'
sub_tweet = 'you here'

In [22]:
tweet.find(sub_tweet)

21

### Albert Tokenizations

In [2]:
tokenizer = AlbertTokenizer.from_pretrained('/Users/ankur.kumar/Desktop/Personal/Projects/git projects/tweet-sentiment-extraction/models/albert_xl/model_configs/xxlarge/30k-clean.model')

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [8]:
tweet = 'Sooo SAD I will miss you here in San Diego!!!'
extracted = 'I will miss'
tokenizer.encode(tweet)

[2, 86, 4328, 2950, 31, 129, 1501, 42, 235, 19, 523, 4042, 28116, 3]

In [12]:
tokenizer.encode(extracted)

[2, 31, 129, 1501, 3]

In [11]:
tokenizer.decode([2, 86, 4328, 2950, 31, 129, 1501, 42, 235, 19, 523, 4042, 28116, 3])

'[CLS] sooo sad i will miss you here in san diego!!![SEP]'

In [17]:
tokenizer.tokenize(tweet)

['▁so',
 'oo',
 '▁sad',
 '▁i',
 '▁will',
 '▁miss',
 '▁you',
 '▁here',
 '▁in',
 '▁san',
 '▁diego',
 '!!!']

In [38]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))

[86, 4328, 2950, 31, 129, 1501, 42, 235, 19, 523, 4042, 28116]

In [28]:
tweet.find(extracted)

9

In [30]:
tweet[9:9+len(extracted)]

'I will miss'

In [36]:
index = 0#tweet.find(extracted)
tokenizer.tokenize(tweet[0:index])+tokenizer.tokenize(tweet[index:index+len(extracted)])+tokenizer.tokenize(tweet[index+len(extracted):-1]) 




['▁so',
 'oo',
 '▁sad',
 '▁i',
 '▁will',
 '▁miss',
 '▁you',
 '▁here',
 '▁in',
 '▁san',
 '▁diego',
 '!!']

In [27]:
for word in tweet.split(' '):
    token = tokenizer.tokenize(word)
    print(token)

['▁so', 'oo']
['▁sad']
['▁i']
['▁will']
['▁miss']
['▁you']
['▁here']
['▁in']
['▁san']
['▁diego', '!!!']


In [68]:
tokenizer.special_tokens_map

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '<unk>',
 'sep_token': '[SEP]',
 'pad_token': '<pad>',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [76]:
tokenizer.tokenize('[CLS]'+tweet+'[SEP]'+tweet)

['[CLS]',
 '▁so',
 'oo',
 '▁sad',
 '▁i',
 '▁will',
 '▁miss',
 '▁you',
 '▁here',
 '▁in',
 '▁san',
 '▁diego',
 '!!!',
 '[SEP]',
 '▁so',
 'oo',
 '▁sad',
 '▁i',
 '▁will',
 '▁miss',
 '▁you',
 '▁here',
 '▁in',
 '▁san',
 '▁diego',
 '!!!']

In [95]:
def get_token_start_end(text, extracted_text,extra_text=None, verbose=False):
    
    text = '[CLS] '+text+' [SEP]'
    if extra_text:
        text = text+' '+extra_text+' [SEP]'
    index = text.find(extracted)
    
    first = tokenizer.tokenize(text[0:index])
    middle = tokenizer.tokenize(text[index:index+len(extracted)])
    last = tokenizer.tokenize(text[index+len(extracted):len(text)])
    
    if verbose:
        print(text[0:index], text[index:index+len(extracted)], text[index+len(extracted):len(text)])
    #tokens = ['[CLS]']+first+middle+last+['[SEP]']
    tokens = first+middle+last
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    start = len(first)
    end = len(first)+len(middle)
    
    return token_ids, start, end

In [98]:
tweet = 'Sooo SAD I will miss you here in San Diego!!!'
extracted = 'I will miss you'
token_ids, start, end  = get_token_start_end(tweet, extracted, 'Negative', False)
print(token_ids)
tokenizer.convert_ids_to_tokens(token_ids[start:end])

[2, 86, 4328, 2950, 31, 129, 1501, 42, 235, 19, 523, 4042, 28116, 3, 3682, 3]


['▁i', '▁will', '▁miss', '▁you']

[2, 86, 4328, 2950, 31, 129, 1501, 42, 235, 19, 523, 4042, 28116, 3]